# Описание проекта

Допустим, вы работаете в добывающей компании. Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

<a id='tap_table_of_contents'></a>
## Оглавление
<dl>
    <dt><a href='#plan'>План</a></dt><br/>
    <dt><a href='#opis_dann'>Описание данных</a></dt><br/>
    <dt><a href='#the_task'>Условие задачи</a></dt><br/>
    <dt><a href='#data_preparation'>1. Загрузка и подготовка данных</a></dt><br/>
    <dt><a href='#train_test_model'>2. Обучение и проверка модели</a></dt><br/>
    <dt><a href='#preparation_for_calculating'>3. Подготовка к расчёту прибыли</a></dt><br/>
    <dt><a href='#profit_calculation'>4. Расчёт прибыли и рисков</a></dt><br/>
    <dt><a href='#test_model'>5. Расчет рисков и прибыли для каждого региона</a></dt><br/>
    <dt><a href='#vuvod'>6. Вывод</a></dt><br/>

</dl>

<a id='plan'></a>
## План
    1. Загрузим и подготовим данные. Поясним порядок действий.
    2. Обучим и проверим модель для каждого региона:
      2.1. Разобьем данные на обучающую и валидационную выборки в соотношении 75:25.
      2.2. Обучите модель и сделайте предсказания на валидационной выборке.
      2.3. Сохраним предсказания и правильные ответы на валидационной выборке.
      2.4. Напечатаем на экране средний запас предсказанного сырья и RMSE модели.
      2.5. Проанализируем результаты.
    3. Подготовимся к расчёту прибыли:
      3.1. Все ключевые значения для расчётов сохраним в отдельных переменных.
      3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со           средним запасом в каждом регионе.
      3.3. Напишем выводы по этапу подготовки расчёта прибыли.
    4. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
      4.1. Выберем скважины с максимальными значениями предсказаний.
      4.2. Просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.
      4.3. Рассчитаем прибыль для полученного объёма сырья.
    5. Посчитаем риски и прибыль для каждого региона:
      5.1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
      5.2. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
      5.3. Напишем выводы: предложим регион для разработки скважин и обоснуем выбор.

<a id='data_preparation'></a>
# 1. Загрузка и подготовка данных

<dt><a href='#tap_table_of_contents'>Оглавление</a></dt>

In [10]:
# импортируем необходимые библитотеки 
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from scipy import stats as st

In [11]:
# откроем файлы и запишем их в df_{a,b,c}
df_a = pd.read_csv('/datasets/geo_data_0.csv')
df_b = pd.read_csv('/datasets/geo_data_1.csv')
df_c = pd.read_csv('/datasets/geo_data_2.csv')

# удалим ненужный (мещающий будущему анализу столбeц)
df_a.drop(columns = ['id'], inplace = True)
df_b.drop(columns = ['id'], inplace = True)
df_c.drop(columns = ['id'], inplace = True)

display(df_a.head())
display(df_b.head())
display(df_c.head())

df_a.info()
df_b.info()
df_c.info()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4)
memory usage: 3.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4)
memory usage: 3.1 MB


## Вывод

Загрузили данные(3 таблицы с регионами), в них по 100 000 точек (не 10 000  как написано в условии :) )

<a id='train_test_model'></a>
# 2. Обучение и проверка модели

<dt><a href='#tap_table_of_contents'>Оглавление</a></dt>

In [12]:
# создадим функция для обучения и проверки моделей на основе линейной регрессии
def train_valid_regress(df):
    features = df.drop('product', axis = 1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size = 0.25, random_state = 12345) 
    model = LinearRegression(normalize = True)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    RMSE = (mean_squared_error(target_valid, predicted_valid)) ** 0.5
    MAE = mean_absolute_error(target_valid, predicted_valid)
    r2 = r2_score(target_valid,predicted_valid)
    return RMSE, predicted_valid, target_valid, MAE, r2

# запишем все показатели для 3 месторождений 
RMSE_a, predicted_valid_a, target_valid_a, MAE_a, r2_a = train_valid_regress(df_a)
RMSE_b, predicted_valid_b, target_valid_b, MAE_b, r2_b = train_valid_regress(df_b)
RMSE_c, predicted_valid_c, target_valid_c, MAE_c, r2_c = train_valid_regress(df_c)

# выведем на экран среднее и RMSE
print('средний запас предсказанного сырья в 1 регионе', predicted_valid_a.mean(),'RMSE_a =', RMSE_a)
print('средний запас предсказанного сырья во 2 регионе', predicted_valid_b.mean(),'RMSE_b =', RMSE_b)  
print('средний запас предсказанного сырья в 3 регионе', predicted_valid_c.mean(),'RMSE_c =', RMSE_c) 

средний запас предсказанного сырья в 1 регионе 92.59256778438035 RMSE_a = 37.5794217150813
средний запас предсказанного сырья во 2 регионе 68.72854689544602 RMSE_b = 0.8930992867756158
средний запас предсказанного сырья в 3 регионе 94.96504596800489 RMSE_c = 40.02970873393434


### Вывод

На этом этапе можно сделать довольно необычные выводы, как мы видим запасов в среднем меньше всего во 2 регионе, при этом модель посчитала данное значение с невероятно высокой точностью, относитеьно других регионов, где значения довольно высокие, но `RMSE` тоже не маленькое. Из этих выводов следует что на данном этапе сказать довольно трудно, нужны дальнейшие исследования.

<a id='profit_calculation'></a>
# 3. Подготовка к расчёту прибыли

<dt><a href='#tap_table_of_contents'>Оглавление</a></dt>

In [13]:
# посчитаем необходимый запас сырья
development_budget = 10 ** 10
cost_of_one_thousand_barrel = 450 * (10 **3)
amount_of_points = 200 
required_production_volume = (development_budget) / (cost_of_one_thousand_barrel * amount_of_points) 

print(required_production_volume)

111.11111111111111


### Вывод

Сюдя по рассчетам, мы видим что 2 регион пока меньше других подходит нам. 

<a id='data_preparation'></a>
# 4. Расчёт прибыли и рисков 

<dt><a href='#tap_table_of_contents'>Оглавление</a></dt>

In [15]:
# предсказания в Series
predicted_valid_a = pd.Series(predicted_valid_a)
predicted_valid_b = pd.Series(predicted_valid_b)
predicted_valid_c = pd.Series(predicted_valid_c)

# индексы разбросаны  - сбросим
target_valid_a  = target_valid_a.reset_index(drop=True)
target_valid_b  = target_valid_b.reset_index(drop=True)
target_valid_c = target_valid_c.reset_index(drop=True)


# функция для расчета прибыли
def profit_calculation (target, probabilities):
    probs_sorted = probabilities.sort_values(ascending = False)
    top_200_wells = target[probs_sorted.index].head(200)
    raw_material_volume = top_200_wells.sum()
    profit = (cost_of_one_thousand_barrel * raw_material_volume) - development_budget
    return profit

### Вывод

Обе скважины оказались прибыльными, первая немного(нужно сравнивать с  рисками и точностью модели конечно) первое месторождение более  доходное чем второе.

<a id='data_preparation'></a>
# 5. Расчет рисков и прибыли для каждого региона

<dt><a href='#tap_table_of_contents'>Оглавление</a></dt>

In [16]:
# создадим списки для заполнения
values_a = []
values_b = []
values_c = []
# постоянная рандома
state = np.random.RandomState(12345)

# создадим функцию для Bootstrap
def calculate_risks_and_rewards(target, probabilities, values):
    for i in range(1000):
        target_subsample = target.sample(n = 500, replace = True, random_state = state)
        probs_subsample = probabilities[target_subsample.index]
        values.append(profit_calculation(target_subsample, probs_subsample))

# объявим       
calculate_risks_and_rewards(target_valid_a, predicted_valid_a, values_a)
calculate_risks_and_rewards(target_valid_b, predicted_valid_b, values_b)
calculate_risks_and_rewards(target_valid_c, predicted_valid_c, values_c)

In [29]:
# предсказания в Series
values_a = pd.Series(values_a)
values_b = pd.Series(values_b)
values_c = pd.Series(values_c)

# посчитаем среднюю прибыль
mean_values_a = values_a.mean()
mean_values_b = values_b.mean()
mean_values_c = values_c.mean()

# функция для подсчета возможного процента убыточных ситуаций
def damages(df):
    return (df < 0).mean() * 100.

# объявим
percent_damages_a = damages(values_a)
percent_damages_b = damages(values_b)
percent_damages_c = damages(values_c)

# посчитаем доверительный интервал
values_dict = {'values_a': values_a, 'values_b' : values_b, 'values_c' : values_c}
confidence_dict = {'confidence_a': [], 'confidence_b' : [], 'confidence_c' : []}

# функция для расчета доверительного интервала
def confidence_interval(df):
    for k in ['a', 'b', 'c']:
        x = values_dict[f'values_{k}']
        lower = x.quantile(.05) / 10 ** 9 
        upper = x.quantile(.95) / 10 ** 9
        confidence_dict[f'confidence_{k}'] = [lower, upper]

# прменим функцию        
confidence_interval(confidence_dict)

# выведем все на экран для "a", "b" и "c"
print("Средняя прибыль от 'a' месторождения в млрд р:", mean_values_a / 10 ** 9)
print("95%-ый доверительный интервал от 'a' месторождения:", confidence_dict['confidence_a'])
print("Убыток от 'a' месторождения с вероятностью :", percent_damages_a, '%', '\n')

print("Средняя прибыль от 'b' месторождения в млрд р:", mean_values_b / 10 ** 9)
print("95%-ый доверительный интервал от 'b' месторождения:", confidence_dict['confidence_b'])
print("Убыток от 'b' месторождения с вероятностью :", percent_damages_b, '%', '\n')

print("Средняя прибыль от 'с' месторождения в млрд р:", mean_values_c / 10 ** 9)
print("95%-ый доверительный интервал от 'c' месторождения:", confidence_dict['confidence_c'])
print("Убыток от 'c' месторождения с вероятностью :", percent_damages_c, '%')

Средняя прибыль от 'a' месторождения в млрд р: 0.4259385269105923
95%-ый доверительный интервал от 'a' месторождения: [-0.03180311434611654, 0.881341454360281]
Убыток от 'a' месторождения с вероятностью : 6.0 % 

Средняя прибыль от 'b' месторождения в млрд р: 0.5182594936973249
95%-ый доверительный интервал от 'b' месторождения: [0.18659107459380914, 0.8886405913545532]
Убыток от 'b' месторождения с вероятностью : 0.3 % 

Средняя прибыль от 'с' месторождения в млрд р: 0.42019400534405005
95%-ый доверительный интервал от 'c' месторождения: [-0.0260888562899228, 0.9095978254696507]
Убыток от 'c' месторождения с вероятностью : 6.2 %


### Вывод

Мне кажется самым удачным для разработки регион `b`. Судя по среднему возможному доходу эти регионы `(a, c)` практически идентичны. В то время как регион `b` выбивается в лидеры. Доверительный интервал также совпадает полностью `(a, c)`, вероятность убытков ниже в первом регионе(хоть бы и казалось, что незначительно). Но в регионе `b` этот показатель практически нулевой,  поэтому я бы советовал регион `b`. Все таки в отношении рисков показатели разнятся значительнее(в процентом отношении) чем в потенциальной прибыли в данных регионах. +  Доверительный интервал вообще положительный у региона `b`

<a id='vuvod'></a>
# Общий вывод
    1. Загрузили и подготовили данные. Пояснили порядок действий.
    2. Обучили и проверили модель для каждого региона:
      2.1. Разбили данные на обучающую и валидационную выборки в соотношении 75:25.
      2.2. Обучили модель и сделали предсказания на валидационной выборке.
      2.3. Сохранили предсказания и правильные ответы на валидационной выборке.
      2.4. Напечатали на экране средний запас предсказанного сырья и RMSE модели.
      2.5. Проанализировали результаты.
    3. Подготовились к расчёту прибыли:
      3.1. Все ключевые значения для расчётов сохранили в отдельных переменных.
      3.2. Рассчитали достаточный объём сырья для безубыточной разработки новой скважины. Сравнили полученный объём сырья со           средним запасом в каждом регионе.
      3.3. Написали выводы по этапу подготовки расчёта прибыли.
    4. Написали функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
      4.1. Выбрали скважины с максимальными значениями предсказаний.
      4.2. Просуммировали целевое значение объёма сырья, соответствующее этим предсказаниям.
      4.3. Рассчитали прибыль для полученного объёма сырья.
    5. Посчитали риски и прибыль для каждого региона:
      5.1. Применили технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
      5.2. Нашли среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
      5.3. Написали выводы: предложили регион для разработки скважин и обосновали выбор.
      Выбор пал на регион B ввиду более благоприятных(с точки зрения прогнозирования) условий: очнь низкого процента убытка, самой большой предположительной прибыли. 